Option chains
=======

In [2]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)

<IB connected to 127.0.0.1:7496 clientId=12>

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [10]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

In [16]:
spx = Index('SPX')
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

In [15]:
# trying to understand how Index() works
spx = Index(conId=416904)
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

In [25]:
tums = Index('R')
ib.qualifyContracts(tums)

[Index(conId=13062318, symbol='R', exchange='ICEEU', currency='GBP', localSymbol='R')]

In [12]:
help(Index())

Help on Index in module ib_insync.contract object:

class Index(Contract)
 |  ``Contract(**kwargs)`` can create any contract using keyword
 |  arguments. To simplify working with contracts, there are also more
 |  specialized contracts that take optional positional arguments.
 |  Some examples::
 |  
 |      Contract(conId=270639)
 |      Stock('AMD', 'SMART', 'USD')
 |      Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
 |      Forex('EURUSD')
 |      CFD('IBUS30')
 |      Future('ES', '20180921', 'GLOBEX')
 |      Option('SPY', '20170721', 240, 'C', 'SMART')
 |      Bond(secIdType='ISIN', secId='US03076KAA60')
 |  
 |  Method resolution order:
 |      Index
 |      Contract
 |      ib_insync.objects.Object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, symbol='', exchange='', currency='', **kwargs)
 |      Attribute values can be given positionally or as keyword.
 |      If an attribute is not given it will take its value from the
 |      'def

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [28]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2018, 8, 11, 0, 3, 19, 588666, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=2833.28, lastSize=0, high=2842.2, low=2825.81, close=2853.58, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

Take the current market value of the ticker:

In [32]:
spxValue = ticker.marketPrice()
spxValue

2853.58

The following request fetches a list of option chains:

In [33]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

/Users/lorinfields/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/lorinfields/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"{20180920, 20180816, 20181115, 20181018, 20201...","{2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565...."
1,CBOE,416904,SPXW,100,"{20181019, 20181116, 20181031, 20180822, 20181...","{2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565...."
2,SMART,416904,SPXW,100,"{20181019, 20181116, 20181031, 20180822, 20181...","{2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565...."
3,CBOE,416904,SPX,100,"{20180920, 20180816, 20181115, 20181018, 20201...","{2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565...."


These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the montly options trading on SMART:

In [34]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId=416904, tradingClass='SPX', multiplier='100', expirations={'20180920', '20180816', '20181115', '20181018', '20201217', '20200618', '20190117', '20190620', '20190919', '20190314', '20191219', '20181220'}, strikes={2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565.0, 3080.0, 2570.0, 2060.0, 1550.0, 2575.0, 3600.0, 3090.0, 2580.0, 2070.0, 2585.0, 1050.0, 2075.0, 3100.0, 2590.0, 2080.0, 2595.0, 550.0, 1575.0, 2600.0, 3110.0, 2090.0, 2605.0, 3120.0, 2610.0, 1075.0, 2100.0, 3125.0, 2615.0, 3130.0, 2620.0, 2110.0, 1600.0, 2625.0, 2115.0, 3140.0, 2630.0, 2120.0, 2635.0, 1100.0, 2125.0, 3150.0, 2640.0, 2130.0, 2645.0, 2135.0, 600.0, 1625.0, 2650.0, 2140.0, 2655.0, 2145.0, 100.0, 1125.0, 2150.0, 2660.0, 3175.0, 2665.0, 2155.0, 2670.0, 2160.0, 1650.0, 2675.0, 3700.0, 2165.0, 2680.0, 2170.0, 2685.0, 1150.0, 2175.0, 3200.0, 2690.0, 2180.0, 2695.0, 2185.0, 650.0, 1675.0, 2700.0, 2190.0, 2705.0, 2195.0, 2710.0, 1175.0, 2200.0, 3225.0, 2715.0, 2205.0, 2720.0, 2

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [35]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART')
        for right in rights
        for expiration in expirations
        for strike in strikes]

ib.qualifyContracts(*contracts)
len(contracts)

48

In [36]:
contracts[0]

Option(conId=315460694, symbol='SPX', lastTradeDateOrContractMonth='20180816', strike=2835.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   180817P02835000', tradingClass='SPX')

Now to get the market data for all options in one go:

In [37]:
tickers = ib.reqTickers(*contracts)

tickers[0]

Ticker(contract=Option(conId=315460694, symbol='SPX', lastTradeDateOrContractMonth='20180816', strike=2835.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   180817P02835000', tradingClass='SPX'), time=datetime.datetime(2018, 8, 11, 0, 5, 37, 495213, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=15.3, lastSize=3, volume=3157, high=20.5, low=11.5, close=6.35, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[], modelGreeks=OptionComputation(impliedVol=0.09371853027476638, delta=-0.5367303133417469, optPrice=16.394785328432903, pvDividend=2.9308841315806826, gamma=0.010915202391372862, vega=1.5426629104687895, theta=-0.96855749960598, undPrice=2833.28))

In [51]:
df = util.df(tickers)
df.head(2)
# df.describe()
df.max_columns = None
display(df)

,contract,time,bid,bidSize,ask,askSize,last,lastSize,prevBid,prevBidSize,...,dividends,ticks,tickByTicks,domBids,domAsks,domTicks,bidGreeks,askGreeks,lastGreeks,modelGreeks
0,"Option(conId=315460694, symbol='SPX', lastTrad...",2018-08-11 00:05:37.495213+00:00,-1.0,0,-1.0,0,15.30,3.0,NaN,NaN,...,None,[],[],[],[],[],None,None,None,"(0.09371853027476638, -0.5367303133417469, 16...."
1,"Option(conId=315460697, symbol='SPX', lastTrad...",2018-08-11 00:05:37.489945+00:00,-1.0,0,-1.0,0,18.80,1.0,NaN,NaN,...,None,[],[],[],[],[],None,None,None,"(0.09009466589701971, -0.594433837365995, 18.6..."
2,"Option(conId=315460700, symbol='SPX', lastTrad...",2018-08-11 00:05:37.499227+00:00,-1.0,0,-1.0,0,20.50,1.0,NaN,NaN,...,None,[],[],[],[],[],None,None,None,"(0.08670513907897477, -0.6541849601038551, 21...."
3,"Option(conId=315460704, symbol='SPX', lastTrad...",2018-08-11 00:05:37.500104+00:00,-1.0,0,-1.0,0,21.10,2.0,NaN,NaN,...,None,[],[],[],[],[],None,None,None,"(0.08358961631205745, -0.7139520176625357, 24...."
4,"Option(conId=315460709, symbol='SPX', lastTrad...",2018-08-11 00:05:37.493582+00:00,-1.0,0,-1.0,0,26.00,3.0,NaN,NaN,...,None,[],[],[],[],[],None,None,None,"(0.08079083694787804, -0.7712774252725298, 27...."
5,"Option(conId=315460714, symbol='SPX', lastTrad...",2018-08-11 00:05:37.496760+00:00,-1.0,0,-1.0,0,30.90,1.0,NaN,NaN,...,None,[],[],[],[],[],None,None,None,"(0.07835455642232449, -0.823624253771323, 31.4..."
6,"Option(conId=315460719, symbol='SPX', lastTrad...",2018-08-11 00:05:37.490960+00:00,-1.0,0,-1.0,0,37.50,4.0,NaN,NaN,...,None,[],[],[],[],[],None,None,None,"(0.07632949033743303, -0.8688537149829693, 35...."
7,"Option(conId=315460721, symbol='SPX', lastTrad...",2018-08-11 00:05:37.490631+00:00,-1.0,0,-1.0,0,35.70,6.0,NaN,NaN,...,None,[],[],[],[],[],None,None,None,"(0.07476725938799268, -0.905678530471597, 39.8..."
8,"Option(conId=318688441, symbol='SPX', lastTrad...",2018-08-11 00:05:37.496395+00:00,-1.0,0,-1.0,0,40.50,1.0,NaN,NaN,...,None,[],[],[],[],[],None,None,None,"(0.09517771369909465, -0.5067104448332644, 37...."
9,"Option(conId=318688444, symbol='SPX', lastTrad...",2018-08-11 00:05:37.491312+00:00,-1.0,0,-1.0,0,38.10,5.0,NaN,NaN,...,None,[],[],[],[],[],None,None,None,"(0.09375061368294452, -0.529120410425881, 39.9..."


In [52]:
df.to_csv("fuckDuck.csv")

ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
ERROR:ib_insync.wrapper:Erro

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [10]:
ib.disconnect()